In [3]:
import cv2
import pytesseract

In [2]:
!pip install pytesseract

In [7]:
# Set the path to the Tesseract executable file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 
# ^for this to work, need to install tesseract on laptop

In [38]:
# Define a function to extract numbers from an image
def extract_numbers(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
#     cv2.imshow('hi', image)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Apply thresholding to the image
#     _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

#     # Apply some image processing to improve the OCR accuracy
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#     thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#     thresh = cv2.medianBlur(thresh, 3)


    # Apply OCR to the image
    text = pytesseract.image_to_string(image, config='--psm 6 outputbase digits')
    # print(text)
    # Extract the numbers from the OCR output
    numbers = [int(s) for s in text.split() if s.isdigit()]

    return numbers

In [39]:
# so hear me out, we get 30 images after applying box detection stuff, we could skip 1st 10 rows, but this is basic code
# where 11-20 are maximum marks row, 21-30 are obtained marks row
maximum_marks = []
obtained_marks = []

# Loop over the images and extract the numbers from each image
for i in range(10): 
    image_path = f'result{i+11}.png'
#     print(image_path)
    image_numbers = extract_numbers(image_path)
#     print(image_numbers)
    maximum_marks.extend(image_numbers)

for i in range(10):
    image_path = f'result{i+21}.png'
    image_numbers = extract_numbers(image_path)
    obtained_marks.extend(image_numbers)
    
# Print the list of numbers
print(maximum_marks)
print(obtained_marks)

print("total marks: ", sum(maximum_marks))
print("obtained marks: ", sum(obtained_marks))

[20, 60, 10, 10]
[18, 50, 5, 5]
total marks:  100
obtained marks:  78
